In [ ]:
## COVID Dashboard Final Project
##
## By Juan Cabanela, Luke Hebert, and Dio Lopez Vasquez in Summer 2020.

In [ ]:
# This forces a reload of any external library file if it changes.  
# Useful when developing external libraries since otherwise Jupyter 
# will not re-import any library without restarting the python kernel.

%load_ext autoreload
%autoreload 2

In [ ]:
# Enable ipywidget backend for matplotlib
%matplotlib widget

In [ ]:
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib as mpl
from ipywidgets import Layout
from ipywidgets import widgets

# Prevent memory warnings (we are not consuming extra memory by erasing figures)
mpl.rcParams.update({'figure.max_open_warning': 0})

# Set up nicer style as default
mpl.style.use('seaborn-darkgrid')

# Import COVID IO routines from external python libraries
import COVIDlib.data_IO as COVID_IO
import COVIDlib.dashboard_IO as COVID_Dash
import COVIDlib.dashboard_maps as COVID_Maps

## Define variables of interest below
data_dir = 'our_data/'    # Data directory for the COVID datafiles

## Set defaults for menus here
state1 = "Minnesota"
county1 = "Clay"  # Must be county in state1
state2 = "North Dakota"
county2 = "Cass"  # Must be county in state2

# Shrink the legend size since doing single plots
COVID_Dash.legendsize = 8

In [ ]:
# Load all the dataframes into memory
# Retrieve John Hopkins dataframes and add "rates" of deaths/infections
(JH_state_df, JH_cnty_df) = COVID_IO.PtoCDRDataFrames()
JH_state_df = COVID_Dash.cleanJHdata(JH_state_df)
JH_cnty_df = COVID_Dash.cleanJHdata(JH_cnty_df)

# Construct dictionary of FIPS values by placename
FIPSd = COVID_Dash.build_fipsdict(JH_cnty_df, JH_state_df)
# Build a list of all states to put in menus and then dictionary of lists of counties for each state
StatesList = COVID_Dash.BuildStatesList()
CountiesDict = COVID_Dash.BuildCountiesList(JH_cnty_df, StatesList)
# Update FIPS numbers for the states and countys based on defaults
FIPSstate1 = COVID_Dash.GetFIPS(FIPSd, state1)
FIPScnty1 = COVID_Dash.GetFIPS(FIPSd, state1, county1)
FIPSstate2 = COVID_Dash.GetFIPS(FIPSd, state2)
FIPScnty2 = COVID_Dash.GetFIPS(FIPSd, state2, county2)
# Load variable descriptions
JHVarDict = COVID_Dash.BuildJHVarDict()

# Retrieve Apple Mobility Dataframes
(aapl_cnty_df, aapl_state_df) = COVID_IO.PtoAAPLMobilityDataFrames()
COVID_Dash.cleanAAPLdata(aapl_cnty_df)
COVID_Dash.cleanAAPLdata(aapl_state_df)
# Retrieve Google Mobility Dataframes
(goog_cnty_df, goog_state_df) = COVID_IO.PtoGOOGMobilityDataFrames()
COVID_Dash.cleanGOOGdata(goog_cnty_df)
COVID_Dash.cleanGOOGdata(goog_state_df)
# Load mobility variable descriptions
mobilityVarDict = COVID_Dash.BuildMobilityVarDict()

# Load the Dictionary about the Map Variables
MapVarsDict = COVID_Maps.BuildMapVarDict()

# Retrieve IMHE Dataframes
(summary_df, hospitalization_df) = COVID_IO.PtoIMHEDataFrames()
IMHEVarDict = COVID_Dash.BuildIMHEHospitalizationVarDict()


In [ ]:
##
## Functions to call to respond to Widget changes
##

def county_changed(listnum, value):
    # This function updates FIPS values that need to be changed when the county value changes
    global FIPSd, FIPSstate1, FIPScnty1, FIPSstate2, FIPScnty2
    global state1_drop, state2_drop
    
    # Update appropriate FIPS records
    if (listnum == 1):
        FIPSstate1 = COVID_Dash.GetFIPS(FIPSd, state1_drop.value)
        FIPScnty1 = COVID_Dash.GetFIPS(FIPSd, state1_drop.value, value)
    else:
        FIPSstate2 = COVID_Dash.GetFIPS(FIPSd, state2_drop.value)
        FIPScnty2 = COVID_Dash.GetFIPS(FIPSd, state2_drop.value, value)
    
    # Trigger update of textual information
    refresh_text()
    
    # Trigger the plots to redraw here
    refresh_plot1()
    refresh_plot2()
    refresh_plot3()
    
    return


def county1_changed(change):
    # Update the county 1 value
    county_changed(1, change.new)
    return


def county2_changed(change):
    # Update the county 2 value
    county_changed(2, change.new)
    return


def state_changed(listnum, value):
    # This function updates the appropriate counties list widget
    global county1_drop, county2_drop, CountiesDict
    global state1, state2, county1, county2
    
    # Define dictionary of widgets
    widgets_dict = {1: county1_drop, 2: county2_drop}
    cnty_wdgt = widgets_dict[listnum]
    
    # Update the Corresponding County Widget
    cnty_wdgt.options=CountiesDict[value]
    cnty_wdgt.value=CountiesDict[value][0]
    
    # This change in the county values should trigger appropriate county_changed function    
    return
    
    
def state1_changed(change):
    # Update the state 1 value
    state_changed(1, change.new)
    return
    
    
def state2_changed(change):
    # Update the state 2 value
    state_changed(2, change.new)
    return


def Var1_changed(change):
    # This function handles changes to the epidemiological data time series plot variable
    global FIPSd, FIPSstate1, FIPScnty1, FIPSstate2, FIPScnty2
        
    # Trigger the plot redraw here
    refresh_plot1()
    return


def Var2_changed(change):
    # This function handles changes to the mobility data time series plot variable
    global FIPSd, FIPSstate1, FIPScnty1, FIPSstate2, FIPScnty2
        
    # Trigger the plot redraw here
    refresh_plot2()
    return


def Var3_changed(change):
    # This function handles changes to the mobility data time series plot variable
    global FIPSd, FIPSstate1, FIPSstate2
        
    # Trigger the plot redraw here
    refresh_plot3()
    return


def map_changed(change):
    # Updates the state-level map as needed
    
    # Trigger map redraw here
    refresh_map()
    return
    

def refresh_plot1():
    # This redraws the plot using the John Hopkins epidemiological data
    global JH_state_df, JH_cnty_df, JHVarDict, Var1_drop, Var1_smooth, Var1_log
    global FIPSstate1, FIPScnty1, FIPSstate2, FIPScnty2, fig1, ax1
    
    # Collect states and counties to plot
    if (FIPSstate1 != FIPSstate2):
        states = [FIPSstate1, FIPSstate2]
    else:
        states = [FIPSstate1]
    if (FIPScnty1 != FIPScnty2):
        cntys  = [FIPScnty1, FIPScnty2]
    else:
        cntys  = [FIPScnty1]
    
    # Clear previous plot 
    ax1.clear()
    
    # Set plot options
    if (Var1_log.value == 'Yes'):
        ylog_val=True
    else:
        ylog_val=False
    if (Var1_smooth.value == 'Yes'):
        run_avg_val=7
    else:
        run_avg_val=0
        
    # Plot state data
    COVID_Dash.ts_plot(JH_state_df, Var1_drop.value, states, connectdots=True, running_avg=run_avg_val, ylog=ylog_val, fig=fig1, ax=ax1) 
    
    # Plot county data
    if not JHVarDict[Var1_drop.value]['stateonly']:
        COVID_Dash.ts_plot(JH_cnty_df, Var1_drop.value, cntys, connectdots=True, running_avg=run_avg_val, ylog=ylog_val, fig=fig1, ax=ax1) 


def refresh_plot2():
    # This redraws the plot using the Apple and Google mobility data
    global aapl_cnty_df, aapl_state_df, goog_cnty_df, goog_state_df
    global mobilityVarDict, Var2_drop, Var2_smooth
    global FIPSstate1, FIPScnty1, FIPSstate2, FIPScnty2, fig2, ax2
    
    # Collect states and counties to plot
    if (FIPSstate1 != FIPSstate2):
        states = [FIPSstate1, FIPSstate2]
    else:
        states = [FIPSstate1]
    if (FIPScnty1 != FIPScnty2):
        cntys  = [FIPScnty1, FIPScnty2]
    else:
        cntys  = [FIPScnty1]
    
    # Clear previous plot 
    ax2.clear()
    
    # Set plot options
    if (Var2_smooth.value == 'Yes'):
        run_avg_val=7
    else:
        run_avg_val=0
        
    # Plot state and county data
    if (mobilityVarDict[Var2_drop.value]['df'] == 'apple'):
        COVID_Dash.ts_plot(aapl_state_df, Var2_drop.value, states, connectdots=True, running_avg=run_avg_val, fig=fig2, ax=ax2) 
        COVID_Dash.ts_plot(aapl_cnty_df, Var2_drop.value, cntys, connectdots=True, running_avg=run_avg_val, fig=fig2, ax=ax2) 
    else:
        COVID_Dash.ts_plot(goog_state_df, Var2_drop.value, states, connectdots=True, running_avg=run_avg_val, fig=fig2, ax=ax2) 
        COVID_Dash.ts_plot(goog_cnty_df, Var2_drop.value, cntys, connectdots=True, running_avg=run_avg_val, fig=fig2, ax=ax2) 

        
def refresh_plot3():
    # This redraws the plot using the IMHE Predictions
    global hospitalization_df, summary_df, IMHEVarDict, Var3_drop, Var3_log, fig3, ax3
    global FIPSstate1, FIPSstate2
    
    # Collect states to plot
    if (FIPSstate1 != FIPSstate2):
        states = [FIPSstate1, FIPSstate2]
    else:
        states = [FIPSstate1]
    
    # Clear previous plot 
    ax3.clear()
    
    # Set plot options
    if (Var3_log.value == 'Yes'):
        ylog_val=True
    else:
        ylog_val=False
        
    # Plot state IMHE data
    COVID_Dash.ts_plot_Hos(hospitalization_df, Var3_drop.value, states, sum_dataframe=summary_df, connectdots=True, ylog=ylog_val, fig=fig3, ax=ax3) 


def refresh_text():
    global JH_state_df, JH_cnty_df, summary_df, FIPSstate1, FIPScnty1, FIPSstate2, FIPScnty2
    
    # Get textual status
    html_state1 = COVID_Dash.html_status(JH_state_df, FIPSstate1, summary_df, Predictions=True, BedsStatus=False, Display=False)
    html_cnty1 = COVID_Dash.html_status(JH_cnty_df, FIPScnty1, summary_df, BedsStatus=False, Display=False)
    if (FIPSstate2 != FIPSstate1):
        html_state2 = COVID_Dash.html_status(JH_state_df, FIPSstate2, summary_df, Predictions=True, BedsStatus=False, Display=False)
    else:
        html_state2 = ""
    if (FIPScnty2 != FIPScnty1):
        html_cnty2 = COVID_Dash.html_status(JH_cnty_df, FIPScnty2, summary_df, BedsStatus=False, Display=False)
    else:
        html_cnty2 = ""
    html_out = html_state1 + html_cnty1 + html_state2 + html_cnty2
    
    # Update text field
    text_display.value=html_out
    

def SelectMapDF(MapVarsDict, colname, cntyplot=False):
    # This function selects the dataframe to use in mapping this variable
    global JH_state_df, JH_cnty_df, aapl_state_df, aapl_cnty_df, goog_state_df, goog_cnty_df
    
    # Hardcode list of dataframes in order of JH, Apple, and Google, state first then county
    maps_df_list = [JH_state_df, aapl_state_df, goog_state_df, JH_cnty_df, aapl_cnty_df, goog_cnty_df]

    # Dictionary mapping MapVarsDict dataframes to number in list above
    mapsdict = {'JH': 0, 'apple': 1, 'google': 2}
    
    # This will return the number in the maps list of the appropriate dataframe
    df_num = mapsdict[MapVarsDict[colname]['df']] + len(mapsdict)*cntyplot
    
    return maps_df_list[df_num]
    

def refresh_map():
    # Updates the state-level map as needed
    global MapVarsDict, map_drop, map_res, usa_map, usa_legend, usa_tooltip
    
    # Check if the variable is state only, if so, switch to state-only
    if (MapVarsDict[map_drop.value]['stateonly']):
        # Switch to state-level view
        map_res.value = 'State'
    
    # Figure out appropriate dataframe and load it
    if (map_res.value == 'State'):
        hires = False
    else:
        hires = True
    map_df = SelectMapDF(MapVarsDict, map_drop.value, cntyplot=hires)
    # Update using this dataframe
    if (map_res.value == 'State'):
        COVID_Maps.update_us_statesmap(map_df, map_drop.value, usa_map, usa_legend, usa_tooltip)
    else:
        COVID_Maps.update_us_cntymap(map_df, map_drop.value, usa_map, usa_legend, usa_tooltip)
    

In [ ]:
##
## Build Widgets to Display
##

##
## LOCATION CONTROL WIDGETS
##
# Build Dropdown Widgets for first and second county/state combinations with hard-coded defaults for Clay and Cass County in Minnesota
loc_reason = widgets.HTML("<p style='line-height: 1.1em; text-align: justify;'>This Dashboard was designed to allow easy comparison of the two counties making up the Fargo-Moorhead metro area, but it can compare any two counties in the nation.</p>",
                           layout=widgets.Layout(width='325px', margin='10px 10px 10px 10px'))
loc1_label = widgets.HTML(value='<b style="font-size:120%; text-align: right;">Location #1:</b> ',
                         layout=widgets.Layout(width='50px', margin='10px 10px 10px 10px'))
loc2_label = widgets.HTML(value='<b style="font-size:120%; text-align: right;">Location #2:</b> ',
                         layout=widgets.Layout(width='50px', margin='10px 10px 10px 10px'))
spacer = widgets.HTML(value='&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;', layout=widgets.Layout(width='50px') )
state1_drop = widgets.Dropdown(options=StatesList,
                               value=state1,
                               description='State:',
                               disabled=False, 
                              layout=widgets.Layout(width='300px'))
county1_drop = widgets.Dropdown(options=CountiesDict[state1],
                               value=county1,
                               description='County:',
                               disabled=False, 
                              layout=widgets.Layout(width='300px'))
state2_drop = widgets.Dropdown(options=StatesList,
                               value=state2,
                               description='State:',
                               disabled=False, 
                              layout=widgets.Layout(width='300px'))
county2_drop = widgets.Dropdown(options=CountiesDict[state2],
                               value=county2,
                               description='County:',
                               disabled=False, 
                              layout=widgets.Layout(width='300px'))
# Reset FIPS values
FIPSstate1 = COVID_Dash.GetFIPS(FIPSd, state1_drop.value)
FIPScnty1 = COVID_Dash.GetFIPS(FIPSd, state1_drop.value, county1_drop.value)
FIPSstate2 = COVID_Dash.GetFIPS(FIPSd, state2_drop.value)
FIPScnty2 = COVID_Dash.GetFIPS(FIPSd, state2_drop.value, county2_drop.value)
        
# CONNECTIONS TO FUNCTIONS TO HANDLE LOCATION CHANGES
state1_drop.observe(state1_changed, 'value')
state2_drop.observe(state2_changed, 'value')
county1_drop.observe(county1_changed, 'value')
county2_drop.observe(county2_changed, 'value')

##
## TEXT INFORMATION DISPLAY
##
text_display = widgets.HTML(value='Text Information Field',
                            layout=widgets.Layout(width='600px'))
refresh_text() # Update text field with current information


In [ ]:
##
## CONTROLS FOR DISPLAY OF JOHN HOPKINS EPIDEMIOLOGICAL DATA
##
JHvars_menu = []
JHvars = []
for key in JHVarDict:
    # Build variable name list for Dropdown
    JHvars_menu.append((JHVarDict[key]['descript'], key))
    JHvars.append(key)

# Default settings
default_parameter = "dConfirmed"  # Default to number of confirmed new infections
default_smoothing = "Yes"
default_logarithmic = "No"

# This selects the John Hopkins Epidemiological Time Series Data to show in Display 1
Var1_title = widgets.HTML(value='<b>Epidemiological Data:</b>')
Var1_drop = widgets.Dropdown(options=JHvars_menu,
                               value=default_parameter,
                               disabled=False, 
                               style={'description_width': 'initial'},
                               layout=widgets.Layout(width='400px'))
Var1_smooth = widgets.ToggleButtons(options=['Yes', 'No'],
                                    value=default_smoothing,
                                    description='7 Day Running Average?',
                                    style = {'description_width': 'initial', 'button_width': '50px'},
                                    disabled=False,
                                    button_style='')
Var1_log = widgets.ToggleButtons(options=['Yes', 'No'],
                                    value=default_logarithmic,
                                    description='   Logarithmic?',
                                    style = {'description_width': 'initial', 'button_width': '50px'},
                                    disabled=False,
                                    button_style='')

# CONNECTIONS TO FUNCTIONS TO HANDLE EPIDEMIOLOGICAL VARIABLE CHANGES
Var1_drop.observe(Var1_changed, 'value')
Var1_smooth.observe(Var1_changed, 'value')
Var1_log.observe(Var1_changed, 'value')

In [ ]:
##
## CONTROLS FOR DISPLAY OF APPLE AND GOOGLE MOBILITY DATA
##
mobilityvars_menu = []
mobilityvars = []
for key in mobilityVarDict:
    # Build variable name list for Dropdown
    mobilityvars_menu.append((mobilityVarDict[key]['descript'], key))
    mobilityvars.append(key)
    
# This selects the John Hopkins Epidemiological Time Series Data to show in Display 1
Var2_title = widgets.HTML(value='<b>Mobility Data:</b>')
Var2_drop = widgets.Dropdown(options=mobilityvars_menu,
                               value=mobilityvars[0],
                               disabled=False, 
                               style={'description_width': 'initial'},
                               layout=widgets.Layout(width='400px'))
Var2_smooth = widgets.ToggleButtons(options=['Yes', 'No'],
                                    value='No',
                                    description='7 Day Running Average?',
                                    style = {'description_width': 'initial', 'button_width': '50px'},
                                    disabled=False,
                                    button_style='')

# CONNECTIONS TO FUNCTIONS TO HANDLE EPIDEMIOLOGICAL VARIABLE CHANGES
Var2_drop.observe(Var2_changed, 'value')
Var2_smooth.observe(Var2_changed, 'value')

In [ ]:
##
## CONTROLS FOR DISPLAY OF IMHE HOSPITALIZATION DATA
##
IMHEvars_menu = []
IMHEvars = []
for key in IMHEVarDict:
    # Build variable name list for Dropdown
    IMHEvars_menu.append((IMHEVarDict[key], key))
    IMHEvars.append(key)
    
# This selects the John Hopkins Epidemiological Time Series Data to show in Display 1
Var3_title = widgets.HTML(value='<b>IMHE Predictions:</b>')
Var3_drop = widgets.Dropdown(options=IMHEvars_menu,
                               value=IMHEvars[0],
                               disabled=False, 
                               style={'description_width': 'initial'},
                               layout=widgets.Layout(width='400px'))
Var3_log = widgets.ToggleButtons(options=['Yes', 'No'],
                                    value='No',
                                    description='   Logarithmic?',
                                    style = {'description_width': 'initial', 'button_width': '50px'},
                                    disabled=False,
                                    button_style='')

# CONNECTIONS TO FUNCTIONS TO HANDLE EPIDEMIOLOGICAL VARIABLE CHANGES
Var3_drop.observe(Var3_changed, 'value')
Var3_log.observe(Var3_changed, 'value')

In [ ]:
# GENERATE THE US MAPS AND CONTROLS

# Build Map variables menu
MapVars_menu = []
MapVars = []
for key in MapVarsDict:
    # Build variable name list for Dropdown
    MapVars_menu.append((MapVarsDict[key]['descript'], key))
    MapVars.append(key)

default_parameter = "dConfirmed"  # Default to number of confirmed new infections

# Build Map Variable Control
map_title = widgets.HTML(value='<b>Display Most Recent Value of:</b>')
map_drop = widgets.Dropdown(options=MapVars_menu,
                               value=default_parameter,
                               disabled=False, 
                               layout=widgets.Layout(width='300px'))

state_res = "State"
cnty_res = "County (slow to update)"
map_res = widgets.ToggleButtons(options=[state_res, cnty_res],
                             description='Map Resolution:',
                             value='State',
                             disabled=False,
                             button_style='', # 'success', 'info', 'warning', 'danger' or ''
                             tooltips=['Use State-level Data', 'Use County-level Data (Data-intense, Slow to Update)'],
                             style = {'description_width': 'initial', 'button_width': '200px'},
                             layout=widgets.Layout(width='600px'))

# CONNECTIONS TO FUNCTIONS TO HANDLE MAP VARIABLE CHANGES
map_drop.observe(map_changed, 'value')
map_res.observe(map_changed, 'value')

In [ ]:
widgets.HTML("<p style='font-size:8; line-height: 1em; text-align: justify;'>This dashboard is optimized for a 1200 pixel wide display and probably won't look great on a phone.</p><hr>")

In [ ]:
## BUILD FULL INTERFACE:
## Build the display from widgets using a series of HBox (Horizontal boxes) and VBox (Vertical boxes)
## to position the widgets relative to each other.

# ADD A LINE FOR USA STATUS (THAT WON'T NEED TO UPDATE DURING THE RUN)
title_block = widgets.HTML("<b style='font-size: 200%;'>Dragon COVID-19 Dashboard</b>"+COVID_Dash.html_status(JH_state_df, 0, summary_df, BedsStatus=False, Display=False),
                           layout=widgets.Layout(width='600px', margin='10px 10px 10px 10px'))
# Create an initial Matplotlib subplot within output
outputUSA = widgets.Output(layout=widgets.Layout(width='500px', margin='10px 10px 10px 10px'))
with outputUSA:
        fig0, ax0 = plt.subplots(constrained_layout=True, figsize=(5, 2))   
        fig0.canvas.toolbar_visible = False
        fig0.canvas.header_visible = False
        fig0.canvas.footer_visible = False
        fig0.canvas.resizable = False
        fig0.canvas.capture_scroll = False
COVID_Dash.ts_barplot(JH_state_df, 'dConfirmed', 0, ylog=False, running_avg=7, fig=fig0, ax=ax0)
# Shrink font on this (small) plot
ax0.title.set_fontsize(8)
for item in ([ax0.xaxis.label, ax0.yaxis.label] +
              ax0.get_xticklabels() + ax0.get_yticklabels()):
    item.set_fontsize(6)

row0 = widgets.HBox([spacer, title_block, outputUSA, spacer],
                    layout=widgets.Layout(justify_content='space-between', align_content='stretch', align_items='stretch', width='1200px', border='solid 2px',
                                          margin='10px 0px 10px 0px'))


# LOCATION CONTROLS
loc1_controls = widgets.VBox([state1_drop, county1_drop], 
                             layout=widgets.Layout(align_content='center', align_items='center', width='350px'))
loc2_controls = widgets.VBox([state2_drop, county2_drop], 
                             layout=widgets.Layout(align_content='center', align_items='center', width='350px'))
# Generate "Box" for containing the location-control widgets
row1 = widgets.HBox([loc_reason, spacer, loc1_label, loc1_controls, spacer, loc2_label, loc2_controls], 
                    layout=widgets.Layout(width='1200px', border='solid 2px', align_content='center', align_items='center', margin='10px 0px 10px 0px'))

# BUILD WIDGET FOR TEXTBOX and EPIDEMIOLOGIAL PLOTS IN TOP ROW

# Create an initial Matplotlib plot for this row
output1 = widgets.Output()
with output1:
        fig1, ax1 = plt.subplots(constrained_layout=True, figsize=(5, 4))   
        fig1.canvas.toolbar_visible = False
        fig1.canvas.header_visible = False
        fig1.canvas.footer_visible = False
        fig1.canvas.resizable = False
        fig1.canvas.capture_scroll = False
refresh_plot1() # Update this plot
credit1_html = "<p style='font-size:8; line-height: 1em;text-align: justify;'><b>Data Credit:</b> <em>"
credit1_html += COVID_Dash.creditForJH(Display=False)
credit1_html += "</p>"
plot1_credit = widgets.HTML(credit1_html, layout=widgets.Layout(margin='0px 0px 30px 0px', width='500px'))

# Generate "Boxes" for containing the widgets
JHplotContainer  = widgets.VBox([widgets.HBox([Var1_title, Var1_drop]), 
                                 widgets.HBox([Var1_smooth, Var1_log]), 
                                 output1, plot1_credit], 
                                layout=widgets.Layout(align_content='center', align_items='center',
                                                      width='600px',overflow_x='hidden', overflow_y='hidden'))
row2 = widgets.HBox([text_display, JHplotContainer],
                    layout=widgets.Layout(margin='10px 0px 10px 0px'))

# BUILD WIDGET FOR PLOTS IN MIDDLE ROW

# Create an initial Matplotlib plots for this row
output2 = widgets.Output()
with output2:
        fig2, ax2 = plt.subplots(constrained_layout=True, figsize=(5, 4))   
        fig2.canvas.toolbar_visible = False
        fig2.canvas.header_visible = False
        fig2.canvas.footer_visible = False
        fig2.canvas.resizable = False
        fig2.canvas.capture_scroll = False
refresh_plot2() # Update this plot
credit2_html = "<p style='font-size:8; line-height: 1em;text-align: justify;'><b>Data Credit:</b> <em>"
credit2_html += COVID_Dash.creditForApplMob(Display=False)+" "+COVID_Dash.creditForGoogMob(Display=False)
credit2_html += "</p>"
plot2_credit = widgets.HTML(credit2_html, layout=widgets.Layout(margin='0px 0px 30px 0px', width='500px'))

output3 = widgets.Output()
with output3:
        fig3, ax3 = plt.subplots(constrained_layout=True, figsize=(5, 4))   
        fig3.canvas.toolbar_visible = False
        fig3.canvas.header_visible = False
        fig3.canvas.footer_visible = False
        fig3.canvas.resizable = False
        fig3.canvas.capture_scroll = False
refresh_plot3() # Update this plot
credit3_html = "<p style='font-size:8; line-height: 1em;text-align: justify;'><b>Data Credit:</b> <em>"
credit3_html += COVID_Dash.creditForIHME(Display=False)
credit3_html += "</p>"
plot3_credit = widgets.HTML(credit3_html, layout=widgets.Layout(margin='0px 0px 30px 0px', width='500px'))

# Generate "Boxes" for containing the widgets
MobilityplotContainer  = widgets.VBox([widgets.HBox([Var2_title, Var2_drop]), 
                                 widgets.HBox([Var2_smooth]), 
                                 output2, plot2_credit], 
                             layout=widgets.Layout(width='600px',overflow_x='hidden', overflow_y='hidden'))
IMHEplotContainer  = widgets.VBox([widgets.HBox([Var3_title, Var3_drop]), 
                                   Var3_log, 
                                   output3, plot3_credit], 
                                   layout=widgets.Layout(align_content='center', align_items='center',
                                                         width='600px',overflow_x='hidden', overflow_y='hidden'))
row3 = widgets.HBox([MobilityplotContainer, IMHEplotContainer],
                    layout=widgets.Layout(margin='10px 0px 10px 0px'))

# DISPLAY MAP CONTROLS
row4 = widgets.HBox([map_title, map_drop, map_res],
                    layout=widgets.Layout(margin='10px 0px 10px 0px'))

# ACTUALLY DISPLAY THE ENTIRE SETUP
MainDisplay = widgets.VBox([row0, row1, row2, row3, row4], 
                           layout=widgets.Layout(align_content='center', align_items='center',
                                                 margin='0px', width='1200px', 
                                                 overflow_x='hidden', overflow_y='hidden'))
Main = display(MainDisplay)

# BUILD MAP FOR BOTTOM ROW (CAN'T SEEM TO INCLUDE IN MAIN DISPLAY BOXES FOR SOME REASON)
map_df = SelectMapDF(MapVarsDict, map_drop.value, cntyplot=False)
usa_map, usa_legend, usa_tooltip = COVID_Maps.build_us_statesmap(map_df, map_drop.value)
usa_map.layout.width = '1200px'
usa_map.layout.height = '600px'
MapDisplay = display(usa_map)


In [ ]:
mapcredit_html = "<p style='font-size:8; line-height: 1em;text-align: justify;'><b>Data Credit:</b> <em>"
mapcredit_html += COVID_Dash.creditForJH(Display=False)+" "+COVID_Dash.creditForApplMob(Display=False)+" "+COVID_Dash.creditForGoogMob(Display=False)
mapcredit_html += "</p>"
map_credit = widgets.HTML(mapcredit_html, layout=widgets.Layout(align_content='center', align_items='center', margin='0px 0px 0px 0px', width='1200px'))
display(map_credit)

In [ ]:
footer = widgets.HTML("<hr><p style='font-size:8; line-height: 1em;text-align: justify;'>This dashboard gathers data from <a href=\"https://github.com/CSSEGISandData/COVID-19\">John Hopkins</a>, <a href=\"https://www.apple.com/covid19/mobility\">Apple</a>, <a href=\"https://www.google.com/covid19/mobility/\">Google</a>, and the <a href=\"http://www.healthdata.org/covid/data-downloads\">Institute for Health Metrics and Evaluation (IMHE)</a> at the University of Washington allowing a composite view of all four datasets as well as visualizations those sites don't provide. It was constructed as the Software Engineering Final Project for Juan Cabanela, Luke Hebert, and Dionicio Lopez Vasquez in Summer 2020 for the CSIS 340 course at <a href=\"http://mnstate.edu/\">Minnesota State University Moorhead</a>.  Source code for this dashboard is open source and available at <a href=\"https://github.com/JuanCab/COVID_DataViz\">https://github.com/JuanCab/COVID_DataViz</a>.</p>")
display(footer)